# Quantum-classical Mapping

## Code for simulating and infering the classical dynamics

We want to simulate the dynamics

$W\left(s^{\prime} \mid s\right)=W(s \mid s) \delta_{s^{\prime}, s}+\sum_{K=1,2,\dots k} \sum_{a_k} g_{a_k}(s) \delta_{s^{\prime}, F_{a_k}[s]}$

The dirac deltas are there only to tell you which of the states is actually selected, all the calculation is in the flip-probabilities $g_a = [{g_i, g_{ij}}]$. The indices refer to which spin will get flipped. Your data would have the form of $(s_0,s_1, \dots s_t)$ with $s_i$  being states of the system. The $a$ represents a n-spin flip.  $g_i$ is the probability of the flip occurring.

This expression tells us that the transition probability is the sum of all probabilities of all the ways to get into state $s'$ using up to $a$-spin flips. We can use it to calculate the transition rates from a time series. w is an extra parameter that encodes the time scales of the transitions. Not related to coupling between spins.

For fully connected systems we have the flipping parameters:

$g_i(s)=\exp \left[-w_{i i}-s_i\left(h_i+\sum_{i \neq j} J_{i j} s_j\right)\right]$

$g_{ij}(s) = \exp \left(-w_{ij}+J_{ij} s_is_j - s_i \left[\sum_{k} J_{ik} s_k + h_i \right] - s_j \left[ \sum_{k'} J_{jk'} s_{k'} + h_j \right] \right)$

Formalism -1/1 -> 0/1 for binary representation

We set $\beta = 1$ throughout the code

#### Do this first for 2 qubits, then upgrade
2- From time series, learn the weights in W [weights that generated transition matrix] and W directly

3- From W generate eta (ensuring DB; so symmetrize the thing)

5- inverse mapping from rho to W and interpret wyy, wxx terms.wxz [Combine QBM and dynamics simulation code]

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

In [4]:
###  AUXILARY FUNCTIONS
#----------------------------------------------------------------------
@njit
def index_to_spin_state(s_idx, N):
    """Convert an index to a spin state."""
    s = np.zeros((N,), dtype=np.int64)
    for i in range(N):
        # Get the ith bit of s_idx.
        bit = (s_idx >> i) & 1
        # Convert the bit to a spin (-1 or 1).
        s[N - 1 - i] = bit * 2 - 1
    return s

@njit
def spin_state_to_index(s):
    """Convert a spin state to an index."""
    N = len(s)
    s_idx = 0
    for i in range(N):
        # Convert the spin to a bit (0 or 1).
        bit = (s[i] + 1) // 2
        # Set the ith bit of s_idx.
        s_idx |= bit << (N - 1 - i)
    return s_idx

@njit
def bits_flipped_indices(s_idx, flip_index, N):
    """Return the indices of the bits that are flipped when going from s_idx to flip_index."""
    flipped_bits = s_idx ^ flip_index
    flipped_indices = []
    for k in range(N):
        if (flipped_bits & (1 << k)) != 0:
            flipped_indices.append(N - 1 - k)
    return flipped_indices

@njit
def choice(probabilities):
    """A workaround for np.random.choice, which is unsupported by numba"""
    cumulative_distribution = np.cumsum(probabilities)
    return np.searchsorted(cumulative_distribution, np.random.random(), side ="right")


###  SIMULATION FUNCTIONS
#----------------------------------------------------------------------
@njit
def g_single(i, s, w, h, J):
    """Probability associated with a single spin flip"""
    sum_term = h[i]
    for j in range(len(s)):
        if i != j:
            sum_term += J[i, j] * s[j]
    return np.exp(-w[i, i] - s[i] * sum_term)


@njit
def g_double(i, j, s, w, h, J):
    """Probability associated with a double spin flip"""
    sum_term_i = h[i]
    for k in range(len(s)):
        if k != i:
            sum_term_i += J[i, k] * s[k]

    sum_term_j = h[j]
    for k in range(len(s)):
        if k != j:
            sum_term_j += J[j, k] * s[k]

    return np.exp(-w[i, j] + J[i, j] * s[i] * s[j] - s[i] * sum_term_i - s[j] * sum_term_j)

@njit
def spin_flip_parallel(s, w, h, J, no_flip_prob, transition_matrix):
    """
    Flipping algorithm using parallel dynamics using double-spin flips.
    Calculates and stores W when needed.
    """
    N = len(s)  # number of spins in the lattice

    # calculate the index of the current state in the transition matrix
    s_idx = spin_state_to_index(s)

    # update the corresponding row in the transition matrix if it's not yet filled
    if not np.any(transition_matrix[s_idx]):

        # calculate all possible flipping probabilities from state s
        flip_probs = np.zeros((2**N))  # Initialize with zeros

        for i in range(N):
            flip_index = s_idx ^ (1 << i)
            i_flipped = bits_flipped_indices(s_idx, flip_index, N)[0]
            flip_probs[flip_index] = g_single(i_flipped, s, w, h, J)  # single flip probabilities

        for i in range(N):
            for j in range(i+1, N):
                flip_index = s_idx ^ (1 << i) ^ (1 << j)
                i_flipped, j_flipped = bits_flipped_indices(s_idx, flip_index, N)
                flip_probs[flip_index] = g_double(i_flipped, j_flipped, s, w, h, J)  # double flip probabilities

        # normalize the probabilities and add the no-flip probability
        flip_probs_sum = sum(flip_probs)
        flip_probs = np.array([(1 - no_flip_prob) * prob / flip_probs_sum for prob in flip_probs])
        flip_probs[s_idx] = no_flip_prob

        transition_matrix[s_idx] = flip_probs
    else:
        flip_probs = transition_matrix[s_idx]   # get probabilities

    # randomly choose a spin flip according to its probability of occurring
    idx = choice(flip_probs)

    # apply the chosen spin flip
    if idx == s_idx:  # no flip
        pass

    elif idx in [s_idx ^ (1 << i) for i in range(N)]:  # single flip
        i = bits_flipped_indices(s_idx, idx, N)[0]
        s[i] *= -1

    else:  # double flip
        i,j = bits_flipped_indices(s_idx, idx, N)
        s[i] *= -1
        s[j] *= -1

    return s, transition_matrix

@njit
def simulate_dynamics(w, J, h, steps, n_spins, no_flip_prob, fill_missing_entries=True):
    s = np.array([np.random.choice(np.array([-1, 1])) for _ in range(n_spins)], dtype=np.int64) #initial config
    trajectory = np.empty((steps, n_spins), dtype=np.int64)
    # Initialize the transition matrix
    transition_matrix = np.zeros((2**n_spins, 2**n_spins))

    for t in range(steps):
        s, transition_matrix = spin_flip_parallel(s, w, h, J, no_flip_prob, transition_matrix)
        trajectory[t] = s

    # If fill_missing_entries is True, fill in the missing rows in the transition matrix after the simulation
    if fill_missing_entries:
        fill_missing_transition_rows(transition_matrix, w, h, J, no_flip_prob)

    return trajectory, transition_matrix

@njit
def fill_missing_transition_rows(transition_matrix, w, h, J, no_flip_prob):
    """Fill in the missing rows in the transition matrix after the simulation"""
    n_spins = int(np.log2(transition_matrix.shape[0]))
    for s_idx in range(transition_matrix.shape[0]):
        if np.any(transition_matrix[s_idx]):
            continue
        # Convert the index to a spin state
        s = index_to_spin_state(s_idx, n_spins)
        spin_flip_parallel(s, w, h, J, no_flip_prob, transition_matrix)

@njit
def infer_transition_matrix(trajectory):
    """Infers the transition matrix from a time series of the states of the system"""
    n_spins = trajectory.shape[1]
    n_states = 2**n_spins

    # initialize the transition matrix
    transition_matrix = np.zeros((n_states, n_states))

    # count the transitions from each state to each other state
    for i in range(len(trajectory) - 1):
        # Convert the spin states to indices
        from_idx = spin_state_to_index(trajectory[i])
        to_idx = spin_state_to_index(trajectory[i+1])

        # Update the transition matrix
        transition_matrix[from_idx, to_idx] += 1

    # Normalize each row of the transition matrix
    for i in range(n_states):
        row_sum = transition_matrix[i].sum()
        if row_sum > 0:
            transition_matrix[i] /= row_sum

    return transition_matrix



In [5]:
###  PLOTTING FUNCTIONS
#----------------------------------------------------------------------

def plot_combined_dynamics(trajectory, n_spins):
    fig, (ax1,ax2)= plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot for the system's evolution through states
    states = [(''.join(['0' if spin == -1 else '1' for spin in s])) for s in trajectory]  # convert spins to binary
    unique_states = sorted(set(states))
    state_indices = [unique_states.index(state) for state in states]
    
    ax1.step(range(len(state_indices)), state_indices)
    ax1.set_yticks(range(len(unique_states)))
    ax1.set_yticklabels(unique_states)
    ax1.set_title("System's evolution through states")
    ax1.set_xlabel('Time step')
    ax1.set_ylabel('State')

    # Plot for the spins' evolution
    offset_labels = []
    y_ticks = []
    for i in range(n_spins):
        spin_trajectory = trajectory[:, i] + i * 3
        ax2.step(range(len(spin_trajectory)), spin_trajectory, label=f'Spin {i+1}')
        offset_labels += ['0', '1']
        y_ticks += [(i*3 - 1), (i*3 +1)]
        
    ax2.set_title("Spins' evolution")
    ax2.set_xlabel('Time step')
    ax2.set_ylabel('Spin value (with offset)')
    ax2.set_yticks(y_ticks)
    ax2.set_yticklabels(offset_labels)
    ax2.legend(loc='best')
    ax2.plot()
    
    plt.tight_layout()
    plt.show()


## Example simulation for N =3

In [6]:
n_spins = 3
w = np.array([[1,1,1],
              [1,1,1],
              [1,2,1]])
J = np.array([[1,1,4],
              [2,1,1],
              [1,1,3]])
h = np.array([0, 3, 0])

steps = 150000
no_flip_prob = 0.1

# SIMULATE SYSTEM
trajectory, W_model = simulate_dynamics(w, J, h, steps, n_spins, no_flip_prob, fill_missing_entries=True)
W_empirical = infer_transition_matrix(trajectory)

# SET PRINT OPTIONS
np.set_printoptions(precision=2, suppress=True)

# PRINT RESULTS
print(f"{W_model}")
print(np.sum(W_model))
print(f"Maximum difference between W_model and W_empirical: {np.max(W_model - W_empirical):.2f}")

# plot_combined_dynamics(trajectory, n_spins)


[[0.1  0.09 0.68 0.09 0.   0.   0.03 0.  ]
 [0.01 0.1  0.01 0.01 0.04 0.02 0.   0.83]
 [0.39 0.05 0.1  0.39 0.   0.   0.02 0.05]
 [0.   0.   0.   0.1  0.   0.01 0.24 0.65]
 [0.09 0.03 0.7  0.   0.1  0.   0.03 0.03]
 [0.   0.   0.   0.01 0.   0.1  0.11 0.79]
 [0.03 0.   0.21 0.58 0.   0.   0.1  0.08]
 [0.   0.   0.02 0.04 0.   0.02 0.83 0.1 ]]
8.0
Maximum difference between W_model and W_empirical: 0.01


In [57]:
###  INFERENCE FUNCTIONS
#----------------------------------------------------------------------
# @njit
def transition_matrix_to_density_matrix(W):
    # calculate steady-state distribution p
    eigenvalues, eigenvectors = np.linalg.eig(W)
    p = np.abs(eigenvectors[:, eigenvalues.argmax()])

    # calculate the matrix A
    sqrt_p_diag = np.diag(np.sqrt(p))
    A = sqrt_p_diag @ W @ sqrt_p_diag

    # symmetrize A to make it Hermitian
    A = 0.5 * (A + np.transpose(np.conjugate(A)))

    # normalize
    rho = A / np.trace(A)
    return rho

# @njit
def density_matrix_to_transition_matrix(rho):
    # calculate steady-state distribution p
    eigenvalues, eigenvectors = np.linalg.eig(rho)
    p_sqrt = np.abs(eigenvectors[:, eigenvalues.argmax()])

    # calculate the inverse square root of p
    inv_sqrt_p_diag = np.diag(1 / p_sqrt)

    # calculate the matrix A
    A = np.real(rho * np.trace(rho))

    # compute the transition matrix W
    W = inv_sqrt_p_diag @ A @ inv_sqrt_p_diag

    # Nbrmalize each row to ensure the sum of each row equals 1
    W /= W.sum(axis=1, keepdims=True)
    return W


[[-0.06  0.02  0.09  0.05 -0.12  0.    0.01  0.  ]
 [-0.12 -0.13 -0.05  0.   -0.11 -0.    0.    0.4 ]
 [-0.02  0.03  0.05  0.31 -0.4   0.   -0.02  0.04]
 [-0.08 -0.   -0.23  0.01  0.   -0.   -0.1   0.41]
 [-0.01 -0.03  0.21  0.   -0.21 -0.    0.02  0.02]
 [-0.   -0.02  0.   -0.01 -0.   -0.21  0.02  0.23]
 [-0.02  0.    0.08  0.25 -0.04 -0.08  0.02 -0.21]
 [ 0.   -0.3  -0.   -0.1  -0.02 -0.29  0.65  0.07]]


In [58]:
## Examples usage

n_spins = 3
w = np.array([[1,1,1],
              [1,1,1],
              [1,2,1]])
J = np.array([[1,1,4],
              [2,1,1],
              [1,1,3]])
h = np.array([0, 3, 0])

steps = 150000
no_flip_prob = 0.1

# SIMULATE SYSTEM
trajectory, W = simulate_dynamics(w, J, h, steps, n_spins, no_flip_prob, fill_missing_entries=True)
rho = transition_matrix_to_density_matrix(W)
W_return = density_matrix_to_transition_matrix(rho)
# print(W)
# print(rho)
# np.sum(np.diag(rho))
print(W-W_return)

[[-0.06  0.02  0.09  0.05 -0.12  0.    0.01  0.  ]
 [-0.12 -0.13 -0.05  0.   -0.11 -0.    0.    0.4 ]
 [-0.02  0.03  0.05  0.31 -0.4   0.   -0.02  0.04]
 [-0.08 -0.   -0.23  0.01  0.   -0.   -0.1   0.41]
 [-0.01 -0.03  0.21  0.   -0.21 -0.    0.02  0.02]
 [-0.   -0.02  0.   -0.01 -0.   -0.21  0.02  0.23]
 [-0.02  0.    0.08  0.25 -0.04 -0.08  0.02 -0.21]
 [ 0.   -0.3  -0.   -0.1  -0.02 -0.29  0.65  0.07]]


Lets just assume these work for now. 

## Code for the 2-qubit QBM

In [80]:
@njit
def expmat(A):
    """
    Computes the exponential of a given matrix `A'.
    """
    A = 0.5 * (A + np.transpose(np.conjugate(A)))
    evals, evecs = np.linalg.eigh(A)
    N = len(evals)
    res = np.zeros((N,N),np.complex64)
    for i in range(N):
        eigenvector = evecs[:,i]
        projector = np.outer(eigenvector,eigenvector.conj())
        res += np.exp(evals[i]) * projector
    return res

@njit
def logmat(A):
    """
    Computes the natural logarithm of a given matrix `A`.
    """
    A = 0.5 * (A + np.transpose(np.conjugate(A)))
    evals, evecs = np.linalg.eigh(A)
    N = len(evals)
    res = np.zeros((N,N),np.complex64)
    for i in range(N):
        eigenvector = evecs[:,i]
        projector = np.outer(eigenvector,eigenvector)
        res += np.log(evals[i]) * projector
    return res

def generate_interaction_matrices():
    """
    Constructs the tensor product of Pauli matrices for each spin and 
    uses them to build the interaction matrices between the two spins.
    """
    pauI = np.array([[1,0],[0,1]],np.complex64)
    pauX = np.array([[0,1],[1,0]],np.complex64)
    pauY = np.array([[0,-1j],[1j,0]],np.complex64)
    pauZ = np.array([[1,0],[0,-1]],np.complex64)
    pau = np.array([pauI, pauX, pauY, pauZ], dtype=np.complex64)

    interactions = np.zeros((4, 4, 4, 4), dtype=np.complex64)
    for k in range(4):
        for kprime in range(4):
            interactions[k, kprime] = np.kron(pau[k], pau[kprime])
    return interactions


@njit
def hamiltonian2spins(J,interactions):
    """
    Calculates the Hamiltonian matrix of a two-spin system 
    """
    H = np.zeros((4,4),np.complex64)
    for k in range(4):
        for kprime in range(4):
            H += J[k,kprime] * interactions[k,kprime]
    return H

@njit
def rho_model(w,interactions):
    """
    Computes the density matrix of a two-spin system using the Hamiltonian
    and the interaction matrices, and then normalizes it. Exact Diagonalization.
    """
    H      = hamiltonian2spins(w,interactions)        #  get hamiltonian matrix
    rho    = expmat(H)                                #  defintion of rho
    Z      = np.real(np.trace(rho))                   #  get Z
    rho   /= Z                                        #  normalize such that Tr[rho] = 1
    return rho

@njit
def observables(rho, interactions):
    """
    Computes the expectation values of the interaction matrices [observables]
    """
    obs = np.zeros((4,4))
    for k in range(4):
        for kprime in range(4):
            obs[k,kprime] = np.real(np.trace(rho@interactions[k,kprime]))
    return obs

@njit
def KL_divergence(eta,rho):
    """
    Calculates the KL divergence between the model and target distribution.
    """
    return np.real(np.trace(eta@(logmat(eta)-logmat(rho))))



@njit
def generate_parameter_matrix(seed):
    np.random.seed(seed)              # set random seed for reproducibility
    w = np.random.randn(4, 4)         # get normal distributed weights
    w[0, 0] = 0
    return w

@njit
def learn_w(w_eta, interactions, lr, maxiter, tol, random_seed):
    """
    Train the model to fit the target distribution eta
    """
    eta  = rho_model(w_eta, interactions)                      # compute density matrix using ED
    obs_clamped  = observables(eta, interactions)          # get QM clamped statistics  
    
    w    = generate_parameter_matrix(random_seed + 1)      # initialize random parameters
    rho  = rho_model(w, interactions)                      # compute density matrix using ED
    obs  = observables(rho, interactions)            # get QM statistics  
    
    it = 0                                                 #  initialize gradient ascent loop
    diff = np.max(np.abs(obs-obs_clamped))
    Wmax_list = np.zeros(maxiter)                          #  initialize values to store
    KL_list   = np.zeros(maxiter)
    
    
    while (diff > tol and it < maxiter): 
        rho  = rho_model(w, interactions)                   # compute density matrix using ED
        obs  = observables(rho, interactions)               # get QM statistics  
        w              += lr * (obs_clamped -  obs)      #  update parameters
        
        diff = np.max(np.abs(obs - obs_clamped))          #  evaluate differences in clamped and model statistics        
        Wmax = np.max(np.abs(w-w_eta))
        
        Wmax_list[it] = Wmax                               #  store values
        KL_list[it]   = KL_divergence(eta,rho)
        it += 1 
    return w, KL_list, Wmax_list, it

## Example usage of QBM

In [81]:
#learning parameters
lr       = 1.2      #  learning rate 
maxiter  = 2**16    #  iterations
tol      = 1e-10    #  tolerance
random_seed = 666

w_eta            = generate_parameter_matrix(random_seed)    
interactions     = generate_interaction_matrices()     #  generate interaction matrices
w,  KL, Wmax, it = learn_w(w_eta, interactions, lr, maxiter, tol, random_seed)


## Combining QBM and dynamics simulation

In [82]:
def generate_w(wx1x2, wy1y2, w1z2z, hx1, hx2, hy1, hy2, hz1, hz2):
    return np.array([
        [0,  hx2, hy2, hz2],
        [hx1, wx1x2, 0,  0],
        [hy1, 0,  wy1y2, 0],
        [hz1, 0,  0,  w1z2z]
    ])

#interaction parameters
wx1x2 = 0
wy1y2 = 0
wz1z2 = 2
hx1   = 0
hx2   = 0
hy1   = 0
hy2   = 0
hz1   = 1
hz2   = 1
    
w_eta = generate_w(wx1x2 ,wy1y2, wz1z2, hx1, hx2, hy1, hy2, hz1, hz2)

We want to learn the density matrix associated with these parameters (?) This densitry matrix we can map back to transition matrix W, which we can use to simulate the dynamics. Why do we need the QBM? We can just immediately translate QM hamiltonian to density matrix?

## DANGER ZONE: 2+ QUBITS

In [ ]:
###Redundant functions

def flip_metropolis(s, J, h, beta, flip_two_spins=False):
    """Flipping algorithm using sequential dynamics and metropolis hashting"""
    s_prime = np.copy(s)
    
    #select spins to be flipped
    if flip_two_spins and np.random.rand() < 0.5:
        flip_indices = np.random.permutation(len(s))[:2]     # selects two spin indices to be flipped
    else:
        flip_indices = np.array([np.random.randint(len(s))]) # selects a single spin index to be flipped
    
    # flip the spins and calculate energy difference
    s_prime[flip_indices] *= -1
    delta_E = energy(s_prime, J, h) - energy(s, J, h)
    
    # accept or reject new state based on metropolis update rule
    if delta_E < 0 or np.random.rand() < np.exp(-beta * delta_E):
        s = s_prime
    
    return s

def infer_parameters_scipy(data, n_spins):
    """Infer the parameters using some Scipy minimzation of the likelihood. Implement QBM later"""
    w = np.random.normal(size=(n_spins, n_spins))
    h = np.random.normal(size=n_spins)
    J = np.random.normal(size=(n_spins, n_spins))

    initial_guess = np.concatenate((w.flatten(), h.flatten(), J.flatten()))
    result = minimize(log_likelihood, initial_guess, args=(data,n_spins), method='BFGS')

    if result.success:
        inferred_params = result.x
        inferred_w = inferred_params[:n_spins**2].reshape(n_spins, n_spins)
        inferred_h = inferred_params[n_spins**2:n_spins**2+n_spins]
        inferred_J = inferred_params[n_spins**2+n_spins:].reshape(n_spins, n_spins)
        return inferred_w, inferred_h, inferred_J
    else:
        raise ValueError(result.message)
    

###CODE TO CHECK CONVERGENCE TO BOLTZMANN

def boltzmann_distribution(J, h):
    n_spins = len(h)
    all_states = np.array([[1 if digit=='0' else -1 for digit in f"{i:0{n_spins}b}"] for i in range(2**n_spins)])
    energies = np.array([energy(state, J, h) for state in all_states])
    probabilities = np.exp(-energies)
    Z = np.sum(probabilities)
    return probabilities / Z

def plot_convergence_to_boltzmann(J, h, trajectory):
    n_spins = len(h)
    boltzmann_probs = boltzmann_distribution(J, h).flatten()

    # count the occurrences of each unique spin state in the trajectory
    state_counts = {}
    for state in trajectory:
        state_tuple = tuple(state)
        if state_tuple in state_counts:
            state_counts[state_tuple] += 1
        else:
            state_counts[state_tuple] = 1

    # calculate the empirical probabilities from the trajectory
    empirical_probs = np.zeros(2**n_spins)
    for idx, state in enumerate(np.array([[1 if digit=='0' else -1 for digit in f"{i:0{n_spins}b}"] for i in range(2**n_spins)])):
        state_tuple = tuple(state)
        if state_tuple in state_counts:
            empirical_probs[idx] = state_counts[state_tuple] / len(trajectory)


    # plot the Boltzmann probabilities and the empirical probabilities
    state_labels = [system_state(state) for state in np.array([[1 if digit=='0' else -1 for digit in f"{i:0{n_spins}b}"] for i in range(2**n_spins)])]
    x = np.arange(len(state_labels))
    width = 0.35

    fig, ax = plt.subplots(figsize=(6, 3))
    rects1 = ax.bar(x - width / 2, boltzmann_probs, width, label='Boltzmann', color='SteelBlue', alpha=0.8)
    rects2 = ax.bar(x + width / 2, empirical_probs, width, label='Empirical', color='Coral', alpha=0.8)

    ax.set_ylabel('Probability')
    ax.set_xlabel('Spin states')
    ax.set_title('Boltzmann vs Empirical Probabilities')
    ax.set_xticks(x)
    ax.set_xticklabels(state_labels)
    ax.legend()

    plt.show()


###OLD TRANSITION MATRIX FUNCTIONS THAT CALCULATE W SEPARATELY. NOW INCORPORATED IN simulate_dynamics(). NOTE THAT THESE STILL HAVE INCORRECT NORMALIZATION
#----------------------------
def transition_probabilities(s, w, h, J):
    """Calculates the row in the transition matrix corresponding to spin s"""
    N = len(s)
    transition_probs = np.zeros(2**N)
    
    for state_idx in range(2**N):
        s_prime = [(state_idx >> i) & 1 for i in reversed(range(N))] # binary representation with reversed order
        s_prime = (1 - np.array(s_prime)) * 2 - 1 # conversion to spin representation
        
        flipped_spins = np.where(s != s_prime)[0] # returns array with the indices of the differing spins 
        
        #calculate and store the transition probability associated with s'
        if len(flipped_spins) == 0:   #no spin flips
            transition_probs[state_idx] = 1.0
        elif len(flipped_spins) == 1: # single spin flip
            i = flipped_spins[0]
            transition_probs[state_idx] = g_single(i, s, w, h, J)
        elif len(flipped_spins) == 2: # double spin flip
            i, j = flipped_spins
            transition_probs[state_idx] = g_double(i, j, s, w, h, J)
        else:
            transition_probs[state_idx] = 0.0
            
    transition_probs /= np.sum(transition_probs)
    
    return transition_probs

def model_transition_matrix(w, h, J):
    """Calculates the entire transition matrix"""
    N = len(w)
    states = [np.array([(i >> j) & 1 for j in reversed(range(N))]) for i in range(2**N)] # binary representation with reversed order
    states = [(1 - state) * 2 - 1 for state in states] # conversion to spin representation
    
    transition_matrix = np.zeros((2**N, 2**N))
    
    for i, s in enumerate(states):
        probs = transition_probabilities(s, w, h, J)
        transition_matrix[i, :] = probs
    
    return transition_matrix

def log_likelihood(params, data, n_spins):
    """Calculates the log-likelihood of the data given the parameters using the transition probabilities"""
    w = params[:n_spins**2].reshape(n_spins, n_spins)
    h = params[n_spins**2:n_spins**2+n_spins]
    J = params[n_spins**2+n_spins:].reshape(n_spins, n_spins)

    log_likelihood = 0
    for i in range(len(data) - 1):
        transition_prob = 
        log_likelihood += np.log(transition_prob)
    
    return -log_likelihood  # we return the negative likelihood because we want to minimize this function